In [1]:
!pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import necessary libraries
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertForMaskedLM, BertTokenizer
from sklearn.model_selection import train_test_split

from datasets import load_metric
import torch
from torch.optim import AdamW
from tqdm.auto import tqdm

In [ ]:
dataset = load_dataset("KaungHtetCho/MedicalQA")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['Description', 'Patient', 'Doctor'],
        num_rows: 496792
    })
})

In [ ]:
seed=42

In [ ]:
# Split the dataset into training and test data (90% train, 10% test)
# reduce_dataset_split = dataset["train"].train_test_split(test_size=0.01, seed=seed)
# train_set = reduce_dataset_split['test']

train_test_split = dataset["train"].train_test_split(test_size=0.1, seed=seed)
test_set = train_test_split['test']

# Further split the test set into validation and test sets (50% validation, 50% test)
val_test_split = test_set.train_test_split(test_size=0.5, seed=seed)
validation_set = val_test_split['train']
test_set = val_test_split['test']

# Update dataset to include these splits
dataset["train"] = train_test_split['train']
dataset["validation"] = validation_set
dataset["test"] = test_set

In [ ]:
dataset["train"]

Dataset({
    features: ['Description', 'Patient', 'Doctor'],
    num_rows: 447112
})

In [ ]:
model_name = "dmis-lab/biobert-base-cased-v1.1"
model = BertForMaskedLM.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:

def process_data_to_model_inputs(batch):
    inputs = tokenizer(batch["Patient"], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(batch["Doctor"], padding="max_length", truncation=True, max_length=512)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["labels"] = outputs.input_ids
    return batch

# Tokenize and align the data in the dataset
dataset = dataset.map(process_data_to_model_inputs, batched=True)

Map:   0%|          | 0/447112 [00:00<?, ? examples/s]

Map:   0%|          | 0/24840 [00:00<?, ? examples/s]

Map:   0%|          | 0/24840 [00:00<?, ? examples/s]

In [ ]:
def collate_fn(batch):
    # Convert list of dicts to dict of lists
    batch = {key: [d[key] for d in batch] for key in batch[0]}

    # Convert list to tensors
    batch["input_ids"] = torch.tensor(batch["input_ids"]).to(device)
    batch["attention_mask"] = torch.tensor(batch["attention_mask"]).to(device)
    batch["labels"] = torch.tensor(batch["labels"]).to(device)

    return batch

In [ ]:
def validate(model, tokenizer, val_loader):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    with torch.no_grad():  # Disable gradient calculation
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    average_loss = total_loss / len(val_loader)
    return average_loss

In [ ]:
# Convert to pytorch tensors and create data loaders
batch_size = 40
train_loader = DataLoader(dataset["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(dataset["validation"], batch_size=batch_size, collate_fn=collate_fn)
test_loader = DataLoader(dataset["test"], batch_size=batch_size, collate_fn=collate_fn)

In [ ]:
def format_time(seconds):
    """Converts seconds to a string of hours, minutes, and seconds"""
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{int(hours)}h {int(minutes)}m {int(seconds)}s"

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import time

# Check for GPU availability
model.to(device)

# Prepare optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define the training loop
num_epoch = 3
prev_val_loss = float('inf')
for epoch in range(num_epoch):  # number of epochs

    start_time = time.time()  # Record the start time of the epoch

    model.train()

    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Calculate the duration of the epoch
    end_time = time.time()
    epoch_duration = end_time - start_time
    formatted_time = format_time(epoch_duration)

    # Compute training and validation loss
    val_loss = validate(model, tokenizer, val_loader)
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f} | Validation Loss: {val_loss:.4f} | Duration: {formatted_time}")

    # Save model and tokenizer after we got better val loss.
    if val_loss < prev_val_loss:
      print(f"{val_loss} < {prev_val_loss}. Saving the model.")
      model.save_pretrained(f"./model_bioBert")
      tokenizer.save_pretrained(f"./model_bioBert")
      prev_val_loss = val_loss


Epoch 1, Loss: 1.5214 | Validation Loss: 1.4739 | Duration: 3h 11m 42s
1.4738710767405045 < inf. Saving the model.
Epoch 2, Loss: 1.4511 | Validation Loss: 1.4540 | Duration: 3h 11m 42s
1.4539967105392482 < 1.4738710767405045. Saving the model.
Epoch 3, Loss: 1.4264 | Validation Loss: 1.4439 | Duration: 3h 11m 42s
1.4439126312636714 < 1.4539967105392482. Saving the model.


In [ ]:
# Save model and tokenizer after each epoch
# model.save_pretrained(f"./model_t5")
# tokenizer.save_pretrained(f"./model_t5")

In [ ]:
model_directory = "model_bioBert"

In [ ]:
save_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/models/"

In [ ]:
!cp -r $model_directory $save_path

In [4]:
# Copy the model if you want to begin the colab from here.
model_directory = "model_bioBert"
save_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/models/" + model_directory
!cp -r $save_path .

In [5]:
# Load the model and tokenizer from a specified epoch
device = "cuda" if torch.cuda.is_available() else "cpu"

model_directory = "./model_bioBert"  # Adjust the path to match where you've saved the model

load_model = BertForMaskedLM.from_pretrained(model_directory)
load_model.to(device)
load_tokenizer = BertTokenizer.from_pretrained(model_directory)
load_model = load_model.eval()

In [ ]:
# from nltk.translate.bleu_score import corpus_bleu

# def generate_answers(batch):
#     inputs = load_tokenizer(batch["Patient"], return_tensors="pt", padding=True, truncation=True, max_length=512)
#     outputs = load_model.generate(
#             input_ids,
#             max_length=1024,
#             temperature=0.4,
#             do_sample=True,
#             top_k=20
#         )
#     batch["model_output"] = load_tokenizer.batch_decode(outputs, skip_special_tokens=True)
#     return batch

# results = test_loader.dataset.map(generate_answers, batched=True, batch_size=8)
# refs = [[x.split()] for x in results["Doctor"]]
# hyp = [x.split() for x in results["model_output"]]

# bleu_score = corpus_bleu(refs, hyp)
# print("BLEU score:", bleu_score)



In [6]:
def answer_question(question):
    load_model.eval()
    with torch.no_grad():
        input_ids = load_tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids.to(device)
        outputs = load_model.generate(
            input_ids,
            max_length=512,
            temperature=0.4,
            do_sample=True,
            top_k=20
        )
        answer = load_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Example
print(answer_question("I am having a fever, and sore throat. What should I do?"))

i am having a fever, and sore throat. what should i do? or sore.. a you throat. throat throat throat throat.. throat infection throat throat....... you.... and. throat.. throat and.....


In [ ]:
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding

In [ ]:
# save_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/models"